
install movie database?


In [ ]:
// TODO: use papa parse to load tsv files from imdb
var fs = require('fs');
var papa = require('papaparse');
var sqlite3 = require('better-sqlite3');
var importer = require('../Core');
var createTables = importer.import('create movie database tables');

var principals = '/Users/briancullinan/Downloads/title.principals.tsv';
var titles = '/Users/briancullinan/Downloads/title.basics.tsv';
var names = '/Users/briancullinan/Downloads/name.basics.tsv';
var movies = './movies.sqlite.db';

var db = new sqlite3(movies, {});

function importSQL(file, cb) {
    if(typeof file === 'string') {
        file = fs.createReadStream(file);
    }
    var prev = 0;
    var count = 0;
    return new Promise(resolve => papa.parse(file, {
        worker: true,
        delimiter: '\t',
        fastMode: true,
        skipEmptyLines: true,
        chunk: function(results) {
            count++;
            if(count === 1 || count > prev + 1000) {
                prev = count;
                console.log(`${count} - ${results.data.length} - ${results.data[1]}`);
            }
            
            return results.data.map(r => cb(r));
        },
        complete: resolve
    }));
}

function insertPrincipals(data) {
    return db.prepare(`
INSERT OR IGNORE INTO principals
    (tconst, ordering, nconst, category, job, characters)
VALUES (?,?,?,?,?,?)`).run(data);
}

function insertTitles(data) {
    data = typeof data[0] !== 'object' ? [data] : data;
    return db.prepare(`
INSERT OR IGNORE INTO titles
    (tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres)
VALUES ${data.map(d => '(?,?,?,?,?,?,?,?,?)').join(',')}`).run([].concat.apply([], data));
}

function insertNames(data) {
    data = typeof data[0] !== 'object' ? [data] : data;
    return db.prepare(`
INSERT OR IGNORE INTO name
    (nconst, primaryName, birthYear, deathYear, primaryProfession, knownForTitles)
VALUES ${data.map(d => '(?,?,?,?,?,?)').join(',')}`).run([].concat.apply([], data));
}


function installDatabase() {
    createTables(db)
    .then(() => importSQL(principals, insertPrincipals))
    .then(() => importSQL(titles, insertTitles))
    .then(() => importSQL(names, insertNames))
        .then(r => console.log(r))
        .catch(e => console.error(e))
        // close the database connection
        .then(() => db.close())

}



/*
params = [[1,2],[3,4],[5,6],[7,8]]

stmt = db.prepare("INSERT INTO test (num1, num2) VALUES (?1,?2)");
stmt.all([params], function(err, res) {
    ...
});
*/

module.exports  = installDatabase


reading notebook /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb[0]
reading notebook /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[0]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[1]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[2]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[3]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[4]
reading notebook /Users/briancullinan/jupytangular2/Utilities/file system.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/file system.ipynb[0] aka list project files
compiling /Users/briancullinan/jupytangular2/Utilities/file system.ipynb[1] aka common ignore paths
reading notebook /Users/briancullinan/jupytangular2/Analytics/budgeting.i

reading notebook /Users/briancullinan/jupytangular2/Utilities/bash.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/child process.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/compress.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/dates.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/diff.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/file system.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/git commands.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/git.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/google calendar data.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/google calendar graphs.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/google calendar.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/google contacts.ipynb
reading notebook /Users/briancullinan/ju

Database {
  inTransaction: false,
  open: false,
  memory: false,
  readonly: false,
  name: './movies.sqlite.db' }

Find cross section of actors and movies?


In [ ]:
var importer = require('../Core');
var levSort = importer.import('sort by levenshtein distance');
var sqlite3 = require('better-sqlite3');
var movies = './movies.sqlite.db';
var db = new sqlite3(movies, {});

function getTitles(t) {
    t = typeof t === 'string' ? [t] : t;
    return db.prepare(`
SELECT *
FROM principals
WHERE tconst IN (${t.map(i => '?').join(',')})`).all(t);
}

function getTitlesByName(n) {
    n = typeof n === 'string' ? [n] : n;
    return db.prepare(`
SELECT *
FROM principals
WHERE nconst IN (${n.map(i => '?').join(',')})`).all(n);
}

function getActorsByTitles(t, n) {
    t = typeof t === 'string' ? [t] : t;
    n = typeof n === 'string' ? [n] : n;
    return db.prepare(`
SELECT *
FROM principals
WHERE nconst IN (${n.map(i => '?').join(',')})
AND tconst IN (${t.map(i => '?').join(',')})`).all(n.concat(t));
}

var assignPromise = r => resolve => getExactName(r.nconst)
    .then(result => Object.assign(r, result))
    .then(resolve)

function getActorTitlesIntersection(n, t) {
    var titles, actors;
    return Promise.resolve()
        .then(() => getTitlesByName(n))
        .then(ts => titles = ts.map(t => t.tconst))
        .then(() => getTitles(t))
        .then(as => getActorsByTitles(titles, as.map(a => a.nconst)))
        .then(r => importer.runAllPromises(r.map(r => assignPromise(r))))
}

function getTitleTitleIntersection(t) {
    var titles = Array.from(arguments);
    return Promise.resolve()
        .then(() => getTitles(t))
        .then(as => getActorsByTitles(titles.slice(1), as.map(a => a.nconst)))
        .then(r => importer.runAllPromises(r.map(r => assignPromise(r))))
}

function getExactName(n) {
    return Promise.resolve()
        .then(() => db.prepare(`SELECT * FROM name WHERE nconst=?`).get(n))
}

function getExactTitle(t) {
    return Promise.resolve()
        .then(() => db.prepare(`SELECT * FROM titles WHERE tconst=?`).get(t))
}

function getTitle(t) {
    var words = t.split(/\s+/ig);
    return Promise.resolve()
        .then(() => db.prepare(`
SELECT * 
FROM titles 
WHERE (${words.map(word => 'primaryTitle LIKE ?').join(' AND ')})
OR (${words.map(word => 'originalTitle LIKE ?').join(' AND ')})
`).all(words.map(w => `%${w}%`).concat(words.map(w => `%${w}%`))))
        .then(results => levSort(results, t, r => r.primaryTitle)[0])
}

// https://v2.sg.media-imdb.com/suggests/w/werner%20herzog.json
//$$.async();
//getTitle('three colors: white')
//getTitleTitleIntersection('tt0111507', 'tt0111495', 'tt0108394')
//getActorTitlesIntersection('nm0001348', 'tt0082694')
//    .then(r => $$.sendResult(r))
//    .catch(e => $$.sendError(e))
    // close the database connection
//    .then(() => db.close())


[ { tconst: 'tt0108394',
    ordering: 2,
    nconst: 'nm0952498',
    category: 'actor',
    job: '\\N',
    characters: '["Karol Karol"]',
    primaryName: 'Zbigniew Zamachowski',
    birthYear: 1961,
    deathYear: '\\N',
    primaryProfession: 'actor,composer,soundtrack',
    knownForTitles: 'tt0108394,tt0111507,tt0228750,tt0111495' },
  { tconst: 'tt0108394',
    ordering: 3,
    nconst: 'nm0000365',
    category: 'actress',
    job: '\\N',
    characters: '["Dominique"]',
    primaryName: 'Julie Delpy',
    birthYear: 1969,
    deathYear: '\\N',
    primaryProfession: 'actress,writer,director',
    knownForTitles: 'tt0111507,tt0112471,tt0381681,tt0841044' },
  { tconst: 'tt0108394',
    ordering: 5,
    nconst: 'nm0001425',
    category: 'director',
    job: '\\N',
    characters: '\\N',
    primaryName: 'Krzysztof Kieslowski',
    birthYear: 1941,
    deathYear: 1996,
    primaryProfession: 'writer,director,actor',
    knownForTitles: 'tt0101765,tt0108394,tt0111495,tt0095467' },

Database {
  inTransaction: false,
  open: false,
  memory: false,
  readonly: false,
  name: './movies.sqlite.db' }

In [3]:
var sqlite3 = require('better-sqlite3');
var movies = './movies.sqlite.db';
var db = new sqlite3(movies, {});

function dropTitles() {
    db.prepare(`DROP INDEX titles_titles`).run();
    db.prepare(`DROP INDEX titles_types`).run();
    db.prepare(`DROP TABLE names`).run();
}

db.prepare(`SELECT COUNT(*) FROM name`).get();


{ 'COUNT(*)': 9371353 }

how to use elastic search from node?


In [ ]:
var elasticsearch = require('elasticsearch')
var client = elasticsearch.Client({
  host: 'localhost:9200'
})

client.search({
  index: 'books',
  type: 'book',
  body: {
    query: {
      multi_match: {
        query: 'express js',
        fields: ['title', 'description']
      }
    }
  }
}).then(function (response) {
  var hits = response.hits.hits
}, function (error) {
  console.trace(error.message)
})

create movie database tables?


In [ ]:
var sqlite3 = require('better-sqlite3');
var movies = './movies.sqlite.db';

function createTables(db) {
    if(!db) {
        db = new sqlite3(movies, {});
    }

    db.prepare(`
CREATE TABLE IF NOT EXISTS principals (
    tconst TEXT NOT NULL,
    ordering INT NOT NULL,
    nconst TEXT NOT NULL,
    category TEXT,
    job TEXT,
    characters TEXT,
    UNIQUE(tconst, ordering),
    PRIMARY KEY (tconst, ordering)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS principles_titles
ON principals (tconst);`).run()
    db.prepare(`
CREATE INDEX IF NOT EXISTS principles_names
ON principals (nconst);`).run()
    
    db.prepare(`
CREATE TABLE IF NOT EXISTS titles (
    tconst TEXT NOT NULL,
    titleType TEXT NOT NULL,
    primaryTitle TEXT,
    originalTitle TEXT,
    isAdult INTEGER,
    startYear INTEGER,
    endYear INTEGER,
    runtimeMinutes INTEGER,
    genres TEXT,
    UNIQUE(tconst, titleType),
    PRIMARY KEY (tconst, titleType)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS titles_titles
ON titles (tconst);`).run()
    db.prepare(`
CREATE INDEX IF NOT EXISTS titles_types
ON titles (tconst, titleType);`).run()
    
    db.prepare(`
CREATE TABLE IF NOT EXISTS akas (
    tconst TEXT NOT NULL,
    ordering INTEGER NOT NULL,
    title TEXT NOT NULL,
    region TEXT NOT NULL,
    language TEXT,
    types TEXT,
    attributes TEXT,
    isOriginalTitle INTEGER,
    UNIQUE(tconst, ordering),
    PRIMARY KEY (tconst, ordering)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS akas_titles
ON akas (tconst);`).run()
    
    db.prepare(`
CREATE TABLE IF NOT EXISTS crew (
    tconst TEXT NOT NULL,
    directors TEXT,
    writers TEXT,
    UNIQUE(tconst),
    PRIMARY KEY (tconst)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS crew_titles
ON crew (tconst);`).run()
    
    db.prepare(`
CREATE TABLE IF NOT EXISTS episode (
    tconst TEXT NOT NULL,
    parentTconst TEXT NOT NULL,
    seasonNumber INTEGER,
    episodeNumber INTEGER,
    UNIQUE(tconst, parentTconst),
    PRIMARY KEY (tconst, parentTconst)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS episode_titles
ON episode (tconst);`).run()
    db.prepare(`
CREATE INDEX IF NOT EXISTS episode_parents
ON episode (tconst, parentTconst);`).run()
    
    db.prepare(`
CREATE TABLE IF NOT EXISTS ratings (
    tconst TEXT NOT NULL,
    averageRating INTEGER,
    numVotes INTEGER,
    UNIQUE(tconst),
    PRIMARY KEY (tconst)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS episode_titles
ON ratings (tconst);`).run()
    
    db.prepare(`
CREATE TABLE IF NOT EXISTS name (
    nconst TEXT NOT NULL,
    primaryName TEXT,
    birthYear INTEGER,
    deathYear INTEGER,
    primaryProfession TEXT,
    knownForTitles TEXT,
    UNIQUE(nconst),
    PRIMARY KEY (nconst)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS name_names
ON name (nconst);`).run()
    
    return Promise.resolve();
}

module.exports = createTables;


TODO: long term use code analysis tools so we don't have to care about cells between languages

TODO: use quilt data for the database

https://github.com/extrabacon/python-shell

Or use ZMQ interface like ijupyter?

TODO: verify with other databases

http://www.omdbapi.com/

https://www.themoviedb.org/documentation/api






movie website?


In [ ]:
const express = require('express')
const cors = require('cors')
const router = express.Router();


const SCAN_ENVIRONMENT = /([A-Z_\[\]]*)\s*[\:-=]+\s*(.*?)\s*(\n|$)/g
const LOADED = {}
const TIMEOUT = 30 * 1000

async function serviceHandler(service, req, res, next) {
  let codeCell = await importer.interpret(service)
  if (!codeCell) {
    return next()
  }
  if (codeCell.language == 'html') {
    return res.send(codeCell.code)
  }
  let serviceFunction = await importer.import(service)
  if (typeof serviceFunction == 'object' && typeof serviceFunction[Object.keys(serviceFunction)[0]] == 'function') {
    serviceFunction = serviceFunction[Object.keys(serviceFunction)[0]]
  }
  return await serviceFunction(req, res, next)
}


function loadRoutes(app, codeCell) {
  let env = {}
  while ((match = SCAN_ENVIRONMENT.exec(codeCell.markdown.join(''))) !== null) {
    env[match[1]] = match[2]
  }
  if (env['ROUTE']) {
    if (env['STATIC']) {
      app.use(env['ROUTE'], express.static(eval(env['STATIC'])))
    }
    if(env['METHOD'] == 'all') {
      router.all(env['ROUTE'], serviceHandler.bind(null, codeCell.questions[0]))
    } else if (env['METHOD'] == 'post') {
      app.post(env['ROUTE'], serviceHandler.bind(null, codeCell.questions[0]))
    } else {
      app.get(env['ROUTE'], serviceHandler.bind(null, codeCell.questions[0]))
    }
  }
}


async function startServer(services) {
  if (typeof services == 'string') {
    services = [services]
  }
  let codeCells = services.map(service => importer.interpret(service)).flat(1)
  var app = express()
  app.use(cors({
    credentials: true,
    origin: '*'
  }))
  app.use(express.json({ limit: '20mb' }))
  for (let i = 0; i < codeCells.length; i++) {
    loadRoutes(app, codeCells[i])
  }
  app.get('/import/:query', async function (req, res, next) {
    let service = req.params['query']
    let codeCell = importer.interpret(service)
    let env = {}
    while ((match = SCAN_ENVIRONMENT.exec(codeCell.markdown.join(''))) !== null) {
      env[match[1]] = match[2]
    }
    if (env['ROUTE'] && LOADED[codeCell.id]) {
      return res.redirect(env['ROUTE'])
    }

    LOADED[codeCell.id] = true
    loadRoutes(app, codeCell)
    return res.redirect(env['ROUTE'])
  })
  let port = process.env.CHAT_PORT || 8080
  let server = app.listen(port, () => console.log('Listening on http://localhost:' + port))

  server.keepAliveTimeout = TIMEOUT;
  server.headersTimeout = TIMEOUT;

}

module.exports = startServer




express handle movie search?

ROUTE = /movies
METHOD = post


In [ ]:
var sqlite3 = require('better-sqlite3');
var movies = './movies.sqlite.db';
var db = new sqlite3(movies, {});

const {safeurl} = importer.import('domain cache tools')
const levSort = importer.import('sort by levenshtein distance');
const selectModel = importer.import('select llm')

async function handleSearch(req, res) {
  let cardsPage = importer.interpret('movie tile').code
  let promptModel = await selectModel(process.env.DEFAULT_MODEL || 'Default')

  let q1 = 'List any related actors:\n' + req.body.content + '\nOnly return the list and nothing else.'
  let a1 = await promptModel(q1)

  let q2 = 'List any related movies:\n' + req.body.content + '\nOnly return the list and nothing else.'
  let a2 = await promptModel(q2)

  let names = (req.body.content + '\n' + req.body.selected.join('\n') + '\n' + a1).split('\n')
    .map(n => n.split(/[\s,\n\*]+|[0-9]+\.\s*/gi).filter(n => n).join(' ')).filter(n => n).slice(0, 10)

  let movies = (req.body.content + '\n' + req.body.selected.join('\n') + '\n' + a1).split('\n')
    .map(n => n.split(/[\s,\n\*]+|[0-9]+\.\s*|\s*\([0-9]+\)\s*/gi).filter(n => n).join(' ')).filter(n => n).slice(0, 10)

  let query = 'SELECT * FROM name WHERE primaryName LIKE ' + names.map(n => '\'%' + n + '%\'').join(' OR primaryName LIKE ') + ' LIMIT 100'
  let actors = db.prepare(query).all()
  actors = levSort(actors, req.body.content, r => r.primaryName)

  let cards = ''
  for(let i = 0; i < actors.length; i++) {
    cards += cardsPage.replaceAll('${TITLE}', actors[i].primaryName)
                      .replaceAll('${ID}', safeurl(actors[i].primaryName))
                      .replaceAll('${TYPE}', actors[i].birthYear)
                      .replaceAll('${YEAR}', actors[i].deathYear)
                      .replaceAll('${LENGTH}', '')
                      .replaceAll('${GENRE}', actors[i].primaryProfession)
  }


  let query2 = 'SELECT * FROM titles WHERE primaryTitle LIKE ' + movies.map(n => '\'%' + n + '%\'').join(' OR primaryTitle LIKE ') + ' LIMIT 100'
  let titles = db.prepare(query2).all()
  titles = levSort(titles, req.body.content, r => r.primaryTitle)

  for(let i = 0; i < titles.length; i++) {
    cards += cardsPage.replaceAll('${TITLE}', titles[i].primaryTitle)
                      .replaceAll('${ID}', safeurl(titles[i].primaryTitle))
                      .replaceAll('${TYPE}', titles[i].titleType)
                      .replaceAll('${YEAR}', titles[i].startYear)
                      .replaceAll('${LENGTH}', titles[i].runtimeMinutes)
                      .replaceAll('${GENRE}', titles[i].genres)
  }

  // TODO: list titles that multiple actors have in common like function above


  if(req.headers.accept == 'application/json') {
    return res.json({ content: result });
  }

  return res.send(cards)
}

module.exports = handleSearch


express list movies database?

ROUTE = /movies


In [ ]:
var sqlite3 = require('better-sqlite3');
var movies = './movies.sqlite.db';
var db = new sqlite3(movies, {});
const {safeurl} = importer.import('domain cache tools')


function listDatabase(req, res) {
  let moviePage = importer.interpret('movie page').code
  let cardsPage = importer.interpret('movie tile').code
  let count = Object.values(db.prepare('SELECT count(*) FROM titles').get())[0]
  let offset = Math.floor(Math.random() * count / 25) * 25
  let titles = db.prepare('SELECT * FROM titles LIMIT 25 OFFSET ' + Math.floor(offset)).all()
  let cards = ''
  for(let i = 0; i < titles.length; i++) {
    cards += cardsPage.replaceAll('${TITLE}', titles[i].primaryTitle)
                      .replaceAll('${ID}', safeurl(titles[i].primaryTitle))
                      .replaceAll('${TYPE}', titles[i].titleType)
                      .replaceAll('${YEAR}', titles[i].startYear)
                      .replaceAll('${LENGTH}', titles[i].runtimeMinutes)
                      .replaceAll('${GENRE}', titles[i].genres)
  }
  let count2 = Object.values(db.prepare('SELECT count(*) FROM name').get())[0]
  let offset2 = Math.floor(Math.random() * count2 / 25) * 25
  let names = db.prepare('SELECT * FROM name LIMIT 25 OFFSET ' + Math.floor(offset2)).all()
  for(let i = 0; i < names.length; i++) {
    cards += cardsPage.replaceAll('${TITLE}', names[i].primaryName)
                      .replaceAll('${ID}', safeurl(names[i].primaryName))
                      .replaceAll('${TYPE}', names[i].birthYear)
                      .replaceAll('${YEAR}', names[i].deathYear)
                      .replaceAll('${LENGTH}', '')
                      .replaceAll('${GENRE}', names[i].primaryProfession)
  }
  moviePage = moviePage.replaceAll('${CARDS}', cards)
  res.send(moviePage)
}

module.exports = listDatabase


movie page?


In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Movie & Actor Tiles</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
            background-color: #f4f4f4;
        }

        .container {
            max-width: 800px;
            margin: auto;
        }

        .search-box {
            width: 100%;
            padding: 10px;
            margin-bottom: 20px;
            font-size: 16px;
            box-sizing: border-box;
        }

        .grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
        }

        .card {
            background: white;
            padding: 15px;
            border-radius: 10px;
            box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);
            text-align: center;
            overflow: hidden;
        }

        .card h2 {
            font-size: 18px;
            margin-bottom: 5px;
        }

        .card p {
            color: gray;
            font-size: 14px;
        }

        .card input {
            height: 1px;
            width: 1px;
            position: relative;
            top:-10px;
            left:10px;
        }

        .card input:checked ~ label {
            color: darkblue;
        }

        #hiddenContainer {
            display:none;
            visibility: hidden;
        }
    </style>
</head>

<body>
    <div class="container">
        <input type="text" class="search-box" id="search" placeholder="Search movies or actors..."
            onkeyup="filterCards()">
        <div class="grid" id="cardContainer">

            ${CARDS}

        </div>
        <div class="grid" id="hiddenContainer"></div>
    </div>

    <script>
        let fetchTimeout
        function filterCards() {
            if(fetchTimeout) {
                clearTimeout(fetchTimeout)
                fetchTimeout = null
            }
            fetchTimeout = setTimeout(fetchCards, 1000)
        }
        document.addEventListener('change', function(event) {
            if (event.target.type === 'checkbox') {
                fetchCards()
            }
        })
        async function fetchCards() {
            let input = document.getElementById('search').value.toLowerCase();
            let result = await fetch(window.location.origin, {
                method: 'POST',
                body: JSON.stringify({ content: input, selected: Array.from(document.querySelectorAll('input:checked')).map(el => el.getAttribute('value')) }),
                headers: {
                    "Content-Type": "application/json",
                }
            })
            let cards = await result.text()
            let hidden = document.getElementById('hiddenContainer')
            let container = document.getElementById('cardContainer')
            hidden.innerHTML = cards
            setTimeout(function () {
                let childCount = container.childNodes.length
                for(let i = childCount - 1; i > 0; i--) {
                    if(container.childNodes[i].querySelector && container.childNodes[i].querySelector('input').checked) {
                        continue
                    }
                    container.removeChild(container.childNodes[i])
                }
                let hiddenCount = hidden.childNodes.length
                for(let i = 0; i < hiddenCount; i++) {
                    if(!hidden.childNodes[0].querySelector) {
                        hidden.childNodes[0].remove()
                        continue
                    }
                    // container already has this element
                    if(container.querySelector('#' + hidden.childNodes[0].querySelector('input').getAttribute('id'))) {
                        hidden.childNodes[0].remove()
                        continue
                    }
                    container.appendChild(hidden.childNodes[0])
                }
            }, 100)
        }
    </script>
</body>

</html>


movie tile?


In [ ]:
<div class="card" data-title="${TITLE}">
    <input type="checkbox" id="${ID}" value="${TITLE}" />
    <label for="${ID}">
        <h2>${TITLE}</h2>
        <p>${TYPE} (${YEAR})</p>
        <p>${LENGTH}</p>
        <p>${GENRE}</p>
    </label>
</div>